# Imports

All the imports will be gradually added in the following cell when they are necessary.

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import scale
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score 

#This import is just to ignore minor warnings for readability
import warnings
warnings.filterwarnings("ignore")

# Loading and inspection of the data 

First, we procede at <b>loading the data and doing a general exploration</b> to see what types of preprocessing we need before applying an estimator and to get an idea of how the dataset is structured. 

In [2]:
train = pd.read_csv("/Users/lorenzo/Desktop/auto-insurance-fall-2017/train_auto.csv")
test = pd.read_csv("/Users/lorenzo/Desktop/auto-insurance-fall-2017/test_auto.csv")

In [3]:
print(train.shape)
train.head()

(8161, 26)


,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,...,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,1,0,0.0,0,60.0,0,11.0,"$67,349",No,$0,...,"$14,230",11,Minivan,yes,"$4,461",2,No,3,18.0,Highly Urban/ Urban
1,2,0,0.0,0,43.0,0,11.0,"$91,449",No,"$257,252",...,"$14,940",1,Minivan,yes,$0,0,No,0,1.0,Highly Urban/ Urban
2,4,0,0.0,0,35.0,1,10.0,"$16,039",No,"$124,191",...,"$4,010",4,z_SUV,no,"$38,690",2,No,3,10.0,Highly Urban/ Urban
3,5,0,0.0,0,51.0,0,14.0,NaN,No,"$306,251",...,"$15,440",7,Minivan,yes,$0,0,No,0,6.0,Highly Urban/ Urban
4,6,0,0.0,0,50.0,0,NaN,"$114,986",No,"$243,925",...,"$18,000",1,z_SUV,no,"$19,217",2,Yes,3,17.0,Highly Urban/ Urban


In [4]:
train.describe()

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,TRAVTIME,TIF,CLM_FREQ,MVR_PTS,CAR_AGE
count,8161.000000,8161.000000,8161.000000,8161.000000,8155.000000,8161.000000,7707.000000,8161.000000,8161.000000,8161.000000,8161.000000,7651.000000
mean,5151.867663,0.263816,1504.324648,0.171057,44.790313,0.721235,10.499286,33.485725,5.351305,0.798554,1.695503,8.328323
std,2978.893962,0.440728,4704.026930,0.511534,8.627589,1.116323,4.092474,15.908333,4.146635,1.158453,2.147112,5.700742
min,1.000000,0.000000,0.000000,0.000000,16.000000,0.000000,0.000000,5.000000,1.000000,0.000000,0.000000,-3.000000
25%,2559.000000,0.000000,0.000000,0.000000,39.000000,0.000000,9.000000,22.000000,1.000000,0.000000,0.000000,1.000000
50%,5133.000000,0.000000,0.000000,0.000000,45.000000,0.000000,11.000000,33.000000,4.000000,0.000000,1.000000,8.000000
75%,7745.000000,1.000000,1036.000000,0.000000,51.000000,1.000000,13.000000,44.000000,7.000000,2.000000,3.000000,12.000000
max,10302.000000,1.000000,107586.136160,4.000000,81.000000,5.000000,23.000000,142.000000,25.000000,5.000000,13.000000,28.000000


In [5]:
print(test.shape)
test.head()

(2141, 26)


,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,...,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,3,NaN,NaN,0,48.0,0,11.0,"$52,881",No,$0,...,"$21,970",1,Van,yes,$0,0,No,2,10.0,Highly Urban/ Urban
1,9,NaN,NaN,1,40.0,1,11.0,"$50,815",Yes,$0,...,"$18,930",6,Minivan,no,"$3,295",1,No,2,1.0,Highly Urban/ Urban
2,10,NaN,NaN,0,44.0,2,12.0,"$43,486",Yes,$0,...,"$5,900",10,z_SUV,no,$0,0,No,0,10.0,z_Highly Rural/ Rural
3,18,NaN,NaN,0,35.0,2,NaN,"$21,204",Yes,$0,...,"$9,230",6,Pickup,no,$0,0,Yes,0,4.0,z_Highly Rural/ Rural
4,21,NaN,NaN,0,59.0,0,12.0,"$87,460",No,$0,...,"$15,420",1,Minivan,yes,"$44,857",2,No,4,1.0,Highly Urban/ Urban


In [6]:
sum(test["TARGET_FLAG"].isna())

2141

In [7]:
sum(test["TARGET_AMT"].isna())

2141

In [8]:
test.describe()

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,TRAVTIME,TIF,CLM_FREQ,MVR_PTS,CAR_AGE
count,2141.000000,0.0,0.0,2141.000000,2140.000000,2141.000000,2047.000000,2141.000000,2141.000000,2141.000000,2141.000000,2012.000000
mean,5150.098552,NaN,NaN,0.162541,45.016822,0.717422,10.379091,33.152265,5.244745,0.808968,1.765997,8.183400
std,2956.329272,NaN,NaN,0.486949,8.525006,1.116579,4.170008,15.722393,3.971026,1.137481,2.203413,5.766263
min,3.000000,NaN,NaN,0.000000,17.000000,0.000000,0.000000,5.000000,1.000000,0.000000,0.000000,0.000000
25%,2632.000000,NaN,NaN,0.000000,39.000000,0.000000,9.000000,22.000000,1.000000,0.000000,0.000000,1.000000
50%,5224.000000,NaN,NaN,0.000000,45.000000,0.000000,11.000000,33.000000,4.000000,0.000000,1.000000,8.000000
75%,7669.000000,NaN,NaN,0.000000,51.000000,1.000000,13.000000,43.000000,7.000000,2.000000,3.000000,12.000000
max,10300.000000,NaN,NaN,3.000000,73.000000,5.000000,19.000000,105.000000,25.000000,5.000000,12.000000,26.000000


In [9]:
#These loops obtain all the columns names for each dataset along with the data type

for i in train.columns:
    print(i," - ", train[i].dtype)
   
print("")
print("Test set columns \n")

for i in test.columns:
    print(i," - ", test[i].dtype)

INDEX  -  int64
TARGET_FLAG  -  int64
TARGET_AMT  -  float64
KIDSDRIV  -  int64
AGE  -  float64
HOMEKIDS  -  int64
YOJ  -  float64
INCOME  -  object
PARENT1  -  object
HOME_VAL  -  object
MSTATUS  -  object
SEX  -  object
EDUCATION  -  object
JOB  -  object
TRAVTIME  -  int64
CAR_USE  -  object
BLUEBOOK  -  object
TIF  -  int64
CAR_TYPE  -  object
RED_CAR  -  object
OLDCLAIM  -  object
CLM_FREQ  -  int64
REVOKED  -  object
MVR_PTS  -  int64
CAR_AGE  -  float64
URBANICITY  -  object

Test set columns 

INDEX  -  int64
TARGET_FLAG  -  float64
TARGET_AMT  -  float64
KIDSDRIV  -  int64
AGE  -  float64
HOMEKIDS  -  int64
YOJ  -  float64
INCOME  -  object
PARENT1  -  object
HOME_VAL  -  object
MSTATUS  -  object
SEX  -  object
EDUCATION  -  object
JOB  -  object
TRAVTIME  -  int64
CAR_USE  -  object
BLUEBOOK  -  object
TIF  -  int64
CAR_TYPE  -  object
RED_CAR  -  object
OLDCLAIM  -  object
CLM_FREQ  -  int64
REVOKED  -  object
MVR_PTS  -  int64
CAR_AGE  -  float64
URBANICITY  -  object


In [10]:
#I used this cell in order to singularly inspect each column without polluting the notebook too much with things 
#that are not worth to be shown 

print(test["INDEX"][:15])
print(train["INDEX"][:15])

0      3
1      9
2     10
3     18
4     21
5     30
6     31
7     37
8     39
9     47
10    60
11    62
12    63
13    64
14    68
Name: INDEX, dtype: int64
0      1
1      2
2      4
3      5
4      6
5      7
6      8
7     11
8     12
9     13
10    14
11    15
12    16
13    17
14    19
Name: INDEX, dtype: int64


# Necessary preprocessing

As we can see, <b>some numerical variables are not considered as numerical </b> so we need to convert them in order to use them to make predictions. At the same time, we separate numerical and categorical columns in order to better process them. It can easily be seen that the single values can be seen as strings which present a "$" symbol as the first character. Also the "," is a problem for the conversion. 

In [11]:
#here we define the numerical and categorical columns. And the target / not target columns.

num_col_convert = ["INCOME", "HOME_VAL", "OLDCLAIM","BLUEBOOK"]
num_col_others = ["KIDSDRIV", "AGE", "HOMEKIDS", "YOJ", "TRAVTIME", "TIF", "CLM_FREQ", "MVR_PTS", "CAR_AGE"]
num_col = num_col_convert + num_col_others

cat_col = ["PARENT1", "MSTATUS", "SEX", "EDUCATION", "JOB", "CAR_USE", "CAR_TYPE", "RED_CAR", "REVOKED", "URBANICITY"]

not_target_col = num_col + cat_col
target_col = ["TARGET_FLAG", "TARGET_AMT"]

In [12]:
#this loop remove the ""$" and "," characters from each value in the columns to convert

for col in num_col_convert:
    for row in range(len(train)):
        train[col][row] = str(train[col][row]).replace("$","")
        train[col][row] = str(train[col][row]).replace(",","")

        
#this loop transform the values of the columns into float values

for col in num_col_convert:
    train[col] = np.array(train[col]).astype(float)
    
train.head()

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,...,BLUEBOOK,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY
0,1,0,0.0,0,60.0,0,11.0,67349.0,No,0.0,...,14230.0,11,Minivan,yes,4461.0,2,No,3,18.0,Highly Urban/ Urban
1,2,0,0.0,0,43.0,0,11.0,91449.0,No,257252.0,...,14940.0,1,Minivan,yes,0.0,0,No,0,1.0,Highly Urban/ Urban
2,4,0,0.0,0,35.0,1,10.0,16039.0,No,124191.0,...,4010.0,4,z_SUV,no,38690.0,2,No,3,10.0,Highly Urban/ Urban
3,5,0,0.0,0,51.0,0,14.0,NaN,No,306251.0,...,15440.0,7,Minivan,yes,0.0,0,No,0,6.0,Highly Urban/ Urban
4,6,0,0.0,0,50.0,0,NaN,114986.0,No,243925.0,...,18000.0,1,z_SUV,no,19217.0,2,Yes,3,17.0,Highly Urban/ Urban


We can see that the columns that we had to convert are now considered as numeric (with float values). We can countercheck printing again the column names again with the type. 

In [13]:
for i in num_col_convert:
    print(i," - ", train[i].dtype)

INCOME  -  float64
HOME_VAL  -  float64
OLDCLAIM  -  float64
BLUEBOOK  -  float64


We repeat now the same process with the test set. 

In [14]:
for col in num_col_convert:
    for row in range(len(test)):
        test[col][row] = str(test[col][row]).replace("$","")
        test[col][row] = str(test[col][row]).replace(",","")

for col in num_col_convert:
    test[col] = np.array(test[col]).astype(float)

In addition, we can see that both test and training set present null values. So, we are going to <b> remove the null values </b> keeping the indexes of the dataset. 
We are also going to <b> normalize the dataset </b>

In [15]:
# These will be needed for the output having a format very similar to the initial one

train_out = train
test_out = test

train_out = train.dropna(axis=0)

n_test = test_out[not_target_col]
n_test = n_test.dropna(axis=0)
n_test = pd.merge(n_test, test_out)
test_out = n_test

In [16]:
#standardization

train[num_col]=scale(train[num_col])
train["TARGET_AMT"]=scale(train["TARGET_AMT"])

test[num_col]=scale(test[num_col])

In [17]:
train.describe()

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,HOME_VAL,TRAVTIME,BLUEBOOK,TIF,OLDCLAIM,CLM_FREQ,MVR_PTS,CAR_AGE
count,8161.000000,8161.000000,8.161000e+03,8.161000e+03,8.155000e+03,8.161000e+03,7.707000e+03,7.716000e+03,7.697000e+03,8.161000e+03,8.161000e+03,8.161000e+03,8.161000e+03,8.161000e+03,8.161000e+03,7.651000e+03
mean,5151.867663,0.263816,2.360295e-17,-1.146070e-15,4.373230e-16,-1.191983e-16,-4.264678e-16,-8.125952e-18,2.609176e-16,-7.169312e-17,-8.059014e-17,-2.446069e-16,-5.048447e-17,3.120487e-16,5.721845e-17,-1.723831e-16
std,2978.893962,0.440728,1.000061e+00,1.000061e+00,1.000061e+00,1.000061e+00,1.000065e+00,1.000065e+00,1.000065e+00,1.000061e+00,1.000061e+00,1.000061e+00,1.000061e+00,1.000061e+00,1.000061e+00,1.000065e+00
min,1.000000,0.000000,-3.198147e-01,-3.344214e-01,-3.337210e+00,-6.461205e-01,-2.565677e+00,-1.301211e+00,-1.199449e+00,-1.790726e+00,-1.687793e+00,-1.049422e+00,-4.599817e-01,-6.893704e-01,-7.897153e-01,-1.987296e+00
25%,2559.000000,0.000000,-3.198147e-01,-3.344214e-01,-6.711803e-01,-6.461205e-01,-3.663758e-01,-7.105608e-01,-1.199449e+00,-7.220385e-01,-7.637169e-01,-1.049422e+00,-4.599817e-01,-6.893704e-01,-7.897153e-01,-1.285587e+00
50%,5133.000000,0.000000,-3.198147e-01,-3.344214e-01,2.430577e-02,-6.461205e-01,1.223578e-01,-1.654438e-01,4.873711e-02,-3.053460e-02,-1.508334e-01,-3.258998e-01,-4.599817e-01,-6.893704e-01,-3.239448e-01,-5.759681e-02
75%,7745.000000,1.000000,-9.956434e-02,-3.344214e-01,7.197918e-01,2.497323e-01,6.110914e-01,5.063718e-01,6.494711e-01,6.609693e-01,6.105199e-01,3.976226e-01,6.824097e-02,1.037176e+00,6.075962e-01,6.441120e-01
max,10302.000000,1.000000,2.255266e+01,7.485673e+00,4.197222e+00,3.833143e+00,3.054760e+00,6.414431e+00,5.657069e+00,6.821640e+00,6.417472e+00,4.738757e+00,6.038775e+00,3.626996e+00,5.265301e+00,3.450947e+00


In [18]:
test.describe()

,INDEX,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,HOME_VAL,TRAVTIME,BLUEBOOK,TIF,OLDCLAIM,CLM_FREQ,MVR_PTS,CAR_AGE
count,2141.000000,0.0,0.0,2.141000e+03,2.140000e+03,2.141000e+03,2.047000e+03,2.016000e+03,2.030000e+03,2.141000e+03,2.141000e+03,2.141000e+03,2.141000e+03,2.141000e+03,2.141000e+03,2.012000e+03
mean,5150.098552,NaN,NaN,7.263897e-16,2.605652e-16,-4.652462e-16,3.037249e-17,-4.832444e-17,1.809308e-16,1.140818e-18,-6.507846e-18,1.007290e-16,2.413348e-16,-3.136212e-16,8.452422e-18,-7.780390e-17
std,2956.329272,NaN,NaN,1.000234e+00,1.000234e+00,1.000234e+00,1.000244e+00,1.000248e+00,1.000246e+00,1.000234e+00,1.000234e+00,1.000234e+00,1.000234e+00,1.000234e+00,1.000234e+00,1.000249e+00
min,3.000000,NaN,NaN,-3.338723e-01,-3.287196e+00,-6.426680e-01,-2.489594e+00,-1.283720e+00,-1.183834e+00,-1.791002e+00,-1.651156e+00,-1.069179e+00,-4.696940e-01,-7.113584e-01,-8.016697e-01,-1.419539e+00
25%,2632.000000,NaN,NaN,-3.338723e-01,-7.059501e-01,-6.426680e-01,-3.307975e-01,-7.343099e-01,-1.183834e+00,-7.094894e-01,-7.800379e-01,-1.069179e+00,-4.696940e-01,-7.113584e-01,-8.016697e-01,-1.246073e+00
50%,5224.000000,NaN,NaN,-3.338723e-01,-1.973765e-03,-6.426680e-01,1.489351e-01,-1.818673e-01,4.344083e-02,-9.686876e-03,-1.535893e-01,-3.135302e-01,-4.696940e-01,-7.113584e-01,-3.477224e-01,-3.181353e-02
75%,7669.000000,NaN,NaN,-3.338723e-01,7.020025e-01,2.531342e-01,6.286676e-01,5.523092e-01,6.446501e-01,6.264972e-01,6.596119e-01,4.421187e-01,8.125675e-02,1.047323e+00,5.601723e-01,6.620491e-01
max,10300.000000,NaN,NaN,5.828374e+00,3.283249e+00,3.836343e+00,2.067865e+00,4.912728e+00,3.987278e+00,4.570839e+00,4.074348e+00,4.976012e+00,5.882822e+00,3.685346e+00,4.645698e+00,3.090568e+00


In [19]:
#we remove the rows with null values and create an incremental index

train = train.dropna(axis=0)
train["index"] = range(0,len(train))
train = train.set_index("index")

new_test = test[not_target_col]
new_test = new_test.dropna(axis=0)
new_test = pd.merge(new_test, test)
test = new_test
test["index"] = range(0,len(test))
test = test.set_index("index")

In [20]:
train.columns

Index(['INDEX', 'TARGET_FLAG', 'TARGET_AMT', 'KIDSDRIV', 'AGE', 'HOMEKIDS',
       'YOJ', 'INCOME', 'PARENT1', 'HOME_VAL', 'MSTATUS', 'SEX', 'EDUCATION',
       'JOB', 'TRAVTIME', 'CAR_USE', 'BLUEBOOK', 'TIF', 'CAR_TYPE', 'RED_CAR',
       'OLDCLAIM', 'CLM_FREQ', 'REVOKED', 'MVR_PTS', 'CAR_AGE', 'URBANICITY'],
      dtype='object')

In [21]:
test.columns

Index(['INCOME', 'HOME_VAL', 'OLDCLAIM', 'BLUEBOOK', 'KIDSDRIV', 'AGE',
       'HOMEKIDS', 'YOJ', 'TRAVTIME', 'TIF', 'CLM_FREQ', 'MVR_PTS', 'CAR_AGE',
       'PARENT1', 'MSTATUS', 'SEX', 'EDUCATION', 'JOB', 'CAR_USE', 'CAR_TYPE',
       'RED_CAR', 'REVOKED', 'URBANICITY', 'INDEX', 'TARGET_FLAG',
       'TARGET_AMT'],
      dtype='object')

In [22]:
#for precision, we modify the disposition of the columns in the training set in order to have the same order of
#the test set (first quantitative variables, then categorical, then index and target variables)

train = train[test.columns]
train.columns

Index(['INCOME', 'HOME_VAL', 'OLDCLAIM', 'BLUEBOOK', 'KIDSDRIV', 'AGE',
       'HOMEKIDS', 'YOJ', 'TRAVTIME', 'TIF', 'CLM_FREQ', 'MVR_PTS', 'CAR_AGE',
       'PARENT1', 'MSTATUS', 'SEX', 'EDUCATION', 'JOB', 'CAR_USE', 'CAR_TYPE',
       'RED_CAR', 'REVOKED', 'URBANICITY', 'INDEX', 'TARGET_FLAG',
       'TARGET_AMT'],
      dtype='object')

Now that we dealt with the numerical columns, we can work on the categorical ones. We will proceed <b> encoding the categorical columns with one hot encoding</b>. In order to use those variables and to not give to different labels a greater magnitude. 

In [23]:
#here we create a dataframes just with the categorical columns and remove those columns from the original datasets

cat_train = train[cat_col]
cat_test = test[cat_col]

train = train.drop(cat_col, axis=1)
test = test.drop(cat_col, axis=1)


#here we perform the one hot encoding

encod = OneHotEncoder(sparse=False)

cat_train_enc = encod.fit_transform(cat_train)
col_encod = encod.get_feature_names(cat_train.columns)
cat_train_enc = pd.DataFrame(cat_train_enc, columns=col_encod)

cat_test_enc = encod.fit_transform(cat_test)
col_encod_t = encod.get_feature_names(cat_test.columns)
cat_test_enc = pd.DataFrame(cat_test_enc, columns=col_encod_t)


#finally we merge the encoded data to the initial datasets

final_train = pd.concat([cat_train_enc, train], axis=1)
final_test = pd.concat([cat_test_enc, test], axis=1)

In [24]:
#we separate predictors and target column(s)

target = final_train[target_col]
final_train = final_train.drop(target_col, axis=1)
final_test = final_test.drop(target_col, axis=1)

At this point, besides using all the variables <b>we could also select just some variables</b> with a feature importance plot conosidering the single model or some other method. 

The preprocessing operations <b> could be implemented inside a ML pipeline </b> along with the chosen model for example creating a function for each operation to insert in the pipeline or using pre-buit functions from sklearn or other packages when possible.

# Model implementation

Now, is necessary to <b> implement different models and tune the hyperparameters </b> in order for example to obtain an high accuracy keeping the running time below a treshold. 

In [25]:
#here we just inspect the number of zeros and ones in the training set to see if is balanced

zeros = len(train[train["TARGET_FLAG"]==0])
ones = len(train[train["TARGET_FLAG"]==1])

print(zeros)
print(ones)

4443
1602


We can affirm that the training set in unbalanced since the number of zeros is way higher than the number of ones. 

In addition, the target variable in the test set is null, so in order to compute a more accurate accuracy we can use the <b>K-fold cross-validation</b>. 

<h3> - Logistic Regression - </h3>

In [26]:
model_log = LogisticRegression()

start = time.time()
model_log.fit(final_train, target["TARGET_FLAG"])
end = time.time()

pred_log = model_log.predict(final_test)
sum(pred_log)

278

In [27]:
pred_tr_log = model_log.predict(final_train)
sum(pred_tr_log)

1073

In [28]:
#The accuracy on the training set for the logistic regression

correct = 0
for i in range(len(train)):
    if train["TARGET_FLAG"][i] == pred_tr_log[i]:
        correct += 1
        
accuracy = (correct / len(train))*100

In [29]:
print("Logistic regression")
print("Accuracy on the training set %.5f " % accuracy,"%")
print("Running time %.5f " % (end - start),"sec")

Logistic regression
Accuracy on the training set 79.00744  %
Running time 0.10356  sec


In [30]:
#In this cell we perform a Kfold cross validation in order to have a more reliable accuracy

kfold = KFold(n_splits=20, random_state=1, shuffle=True)
score = cross_val_score(model_log, final_train, target["TARGET_FLAG"], n_jobs=4, cv=kfold)
print("the cross validation accuracy is %.5f " % (np.mean(score)*100),"+-",(np.std(score)*100),"%")

the cross validation accuracy is 78.62523  +- 2.380912785299125 %


In [31]:
#to get the names of the parameters of the model

model_log.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [32]:
# Peform Randomized Search to tune hyperparameters

param = {'C': uniform(loc=50, scale=100)}
log_rand_search = RandomizedSearchCV(model_log, param_distributions=param, n_iter=20, n_jobs=4, cv=5)

start = time.time()
log_rand_search.fit(final_train, target["TARGET_FLAG"])
end = time.time()
print(f"The best set of parameters is: {log_rand_search.best_params_}")

The best set of parameters is: {'C': 56.95999326355326}


In [33]:
pred_logt = log_rand_search.predict(final_test)
pred_tr_logt = log_rand_search.predict(final_train)

correct = 0
for i in range(len(train)):
    if train["TARGET_FLAG"][i] == pred_tr_logt[i]:
        correct += 1
        
accuracy = (correct / len(train))*100

In [34]:
print("Tuned Logistic regression")
print("Accuracy on the training set %.5f " % accuracy,"%")
print("Running time %.5f " % (end - start),"sec")

Tuned Logistic regression
Accuracy on the training set 78.84202  %
Running time 3.96725  sec


In [35]:
kfold = KFold(n_splits=20, random_state=1, shuffle=True)
score = cross_val_score(log_rand_search, final_train, target["TARGET_FLAG"], n_jobs=4, cv=kfold)
print("the cross validation accuracy is %.5f " % (np.mean(score)*100),"+-",(np.std(score)*100),"%")

the cross validation accuracy is 78.75795  +- 2.2010754820331964 %


<h3> - Gradient Boosting Classifier - </h3>

In [36]:
model_boost = GradientBoostingClassifier()

start = time.time()
model_boost.fit(final_train, target["TARGET_FLAG"])
end = time.time()

pred_b = model_boost.predict(final_test)
pred_tr_b = model_boost.predict(final_train)

In [37]:
correct = 0
for i in range(len(train)):
    if train["TARGET_FLAG"][i] == pred_tr_b[i]:
        correct += 1
        
correct

accuracy = (correct / len(train))*100

In [38]:
print("Gradient Boosting Classifier")
print("Accuracy on the training set %.5f " % accuracy,"%")
print("Running time %.5f " % (end - start),"sec")

Gradient Boosting Classifier
Accuracy on the training set 83.04384  %
Running time 1.84272  sec


In [39]:
kfold = KFold(n_splits=10, random_state=1, shuffle=True)
score = cross_val_score(model_boost, final_train, target["TARGET_FLAG"], n_jobs=4, cv=kfold)
print("the cross validation accuracy is %.5f " % (np.mean(score)*100),"+-",(np.std(score)*100),"%")

the cross validation accuracy is 79.25568  +- 1.42172553057345 %


In [40]:
model_boost.get_params().keys()

dict_keys(['ccp_alpha', 'criterion', 'init', 'learning_rate', 'loss', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_iter_no_change', 'presort', 'random_state', 'subsample', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [41]:
# Peform Randomized Search to tune hyperparameters

param = {'learning_rate': uniform(loc=0.0001, scale=1),
         'n_estimators': randint(50, 150)}

boost_rand_search = RandomizedSearchCV(model_boost, param_distributions=param, n_iter=10, n_jobs=4, cv=5)
start = time.time()
boost_rand_search.fit(final_train, target["TARGET_FLAG"])
end = time.time()
print(f"The best set of parameters is: {boost_rand_search.best_params_}")

The best set of parameters is: {'learning_rate': 0.12438567305345273, 'n_estimators': 135}


In [42]:
pred_bt = boost_rand_search.predict(final_test)
pred_tr_bt = boost_rand_search.predict(final_train)

correct = 0
for i in range(len(train)):
    if train["TARGET_FLAG"][i] == pred_tr_bt[i]:
        correct += 1
        
accuracy = (correct / len(train))*100

In [43]:
print("Tuned Gradient Boosting")
print("Accuracy on the training set %.5f " % accuracy,"%")
print("Running time %.5f " % (end - start),"sec")

Tuned Gradient Boosting
Accuracy on the training set 84.41687  %
Running time 34.38162  sec


In [44]:
kfold = KFold(n_splits=10, random_state=1, shuffle=True)
score = cross_val_score(boost_rand_search, final_train, target["TARGET_FLAG"], n_jobs=4, cv=kfold)
print("the cross validation accuracy is %.5f " % (np.mean(score)*100),"+-",(np.std(score)*100),"%")

the cross validation accuracy is 79.00774  +- 1.3701742326232105 %


Taking into account the Kfold cross validation accuracy, we will use the prediction of the model Gradient boosting model without Random search but with the default parameters (it has both the highest mean and lowest standard deviation). However, since the tuning of the parameters is randomized, if we run again the code, this result might change. 

# Output exportation

Here, we could also automate the exportation according to the best model that we found. For example, renaming the variables and using an IF statement, we could automatically compare the different cross validation scores of the models and so create the column with the predictions in the following cell conditionally on the best performance. 

In [45]:
#we create a column with the predictions 

final_test["FLAG_Prediction"] = pred_b
final_train["FLAG_Prediction"] = pred_tr_b

In [46]:
#here we export in a csv file just the test set along with its predictions

final_test.to_csv(r'/Users/lorenzo/Desktop/auto-insurance-fall-2017/test_predictions.csv',header=True)

In [47]:
#now we reset the index of each set, concatenate those in a single complete dataset set
#we reset again the index for precision changing it with a normal counter
#and export the full dataset (train + test) in a csv file

final_train = final_train.set_index("INDEX")
final_test = final_test.set_index("INDEX")

total_data = pd.concat([final_train, final_test], axis=0)
total_data["index"] = range(0,len(total_data))
total_data = total_data.set_index("index")

total_data.to_csv(r'/Users/lorenzo/Desktop/auto-insurance-fall-2017/total_predictions.csv',header=True)

In [48]:
#this is if we want the csv with the data in the initial format plus, obviously, the predictions

test_out["FLAG_Prediction"] = pred_b
train_out["FLAG_Prediction"] = pred_tr_b

test_out = test_out.set_index("INDEX")
train_out = train_out.set_index("INDEX")

tot = pd.concat([train_out, test_out], axis=0)
tot["index"] = range(0,len(tot))
tot = tot.set_index("index")

tot.to_csv(r'/Users/lorenzo/Desktop/auto-insurance-fall-2017/total_predictions_initial_format.csv',header=True)

In [49]:
tot

,TARGET_FLAG,TARGET_AMT,KIDSDRIV,AGE,HOMEKIDS,YOJ,INCOME,PARENT1,HOME_VAL,MSTATUS,...,TIF,CAR_TYPE,RED_CAR,OLDCLAIM,CLM_FREQ,REVOKED,MVR_PTS,CAR_AGE,URBANICITY,FLAG_Prediction
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0,60.0,0,11.0,67349.0,No,0.0,z_No,...,11,Minivan,yes,4461.0,2,No,3,18.0,Highly Urban/ Urban,0
1,0.0,0.0,0,43.0,0,11.0,91449.0,No,257252.0,z_No,...,1,Minivan,yes,0.0,0,No,0,1.0,Highly Urban/ Urban,0
2,0.0,0.0,0,35.0,1,10.0,16039.0,No,124191.0,Yes,...,4,z_SUV,no,38690.0,2,No,3,10.0,Highly Urban/ Urban,0
3,1.0,2946.0,0,34.0,1,12.0,125301.0,Yes,0.0,z_No,...,1,Sports Car,no,0.0,0,No,0,7.0,Highly Urban/ Urban,1
4,1.0,2501.0,0,34.0,0,10.0,62978.0,No,0.0,z_No,...,1,z_SUV,no,0.0,0,No,0,1.0,Highly Urban/ Urban,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7652,NaN,NaN,0,41.0,0,9.0,126829.0,No,334777.0,Yes,...,4,Minivan,yes,0.0,0,No,2,20.0,Highly Urban/ Urban,0
7653,NaN,NaN,0,46.0,0,0.0,0.0,No,0.0,Yes,...,6,Sports Car,no,20519.0,1,Yes,3,8.0,z_Highly Rural/ Rural,0
7654,NaN,NaN,0,46.0,0,12.0,43501.0,No,134977.0,Yes,...,3,Minivan,no,0.0,0,No,2,1.0,z_Highly Rural/ Rural,0
